# Семинар 6. Элементы функционального программирования

На этом семинаре мы наконец-то перейдем к элементам функционального програм.

**Задание функций**
1. Позиционные и именованные аргументы
2. Args, kwargs
3. Значения по умолчанию (в контексте типов данных)
4. Return и pass
5. Области видимости
6. del

**Элементы функционального программирования**
1. Немного теории
2. Анонимные функции
3. Рекурсия
4. Декораторы
5. Итераторы
6. Генераторы
7. Функции для ФП; functools
8. Обратная сторона лямбды

## Задание функций

В чем разница между параметрами и аргументами?

### Позиционные и именованные параметры

Что знаете про них?

Имейте в виду, что возможен следующий синтаксис:

In [ ]:
def foo(a, b, /, c, d, ∗, e, f):
    print(a, b, c, d, e, f)

### args/kwargs

Вопрос: пример функции, которая принимает неограниченное количество параметров?

Задание: задайте функцию, которая принимает на вход переменное количество аргументов и распечатывает их

In [ ]:
###YOUR CODE###

Что происходит "под капотом"? Аргументы просто распаковываются из списка. `Args`, `kwargs` - просто соглашение, можно назвать как угодно. Обязательно являются последними в списке параметров, поскольку "вбирают в себя" все оставшиеся аргументы.

Вопрос: задайте функцию, которая принимает на вход исключительно именованные аргументы

In [ ]:
###YOUR CODE###

### Изменяемые типы в качестве значений по умолчанию

Рекомендуется избегать значении по умолчанию изменяемых типов. 

Вопрос: сформулируйте, пожалуйста, проблему. 

In [ ]:
def my_function(value, default=[]):
    default.append(value)
    return default

x = my_function(0)
print(x)
y = my_function(1)
print(y)
print(x)

Проблемы можно избежать, если использовать изменяемые типы данных в качестве значений по умолчанию, или использовать уникальный объект для идентификации установленного по умолчанию значения:

In [ ]:
def my_function(
    value,
    default=(UNDEFINED := object())
):

    if default is UNDEFINED: 
        default = []
    default.append(value) 
    return default

x=my_function(0)
y=my_function(1)
print(y)
print(x)

В данном примере UNDEFINED - это уникальный объект в области видимости функции. Если аргумент задается по умолчанию, условие выполняется, и переменная заменяется новым списком, что гарантирует его уникальность.

### Return

Вопрос: что происходит, если не прописываем return?

Обязательно ли return один?

### Pass

Pass - это заглушка, которая делает буквально ничего. Бывает нужна, когда синтаксис требует присутствие, допустим, метода в классе, но никакой функционал за ним мы (пока) не хотим реализовывать. Троеточие `...` является аналогом pass.

In [ ]:
class student:
    pass
def fun():
    pass
def fun2()
    ...

### Область видимости

Область видимости - часть программы, в которой имя переменной может быть использовано.

Области видимости в Питоне выстраиваются в иерархию LEGB: Local, Enclosed, Global, Built-in. Разберемся с этим, памятуя, что 1) переменные определяются в тот момент, когда им присваивается значение 2) функции и методы могут быть вложены в другие функции и методы.

• `Local`: переменая является локальной, если она является параметром функции, или ей присваивается значение в рамках данной функции, и при этом она явным образом не объявлена глобальной или нелокальной.<br>
• `Enclosed` (альтернативно `nonlocal`): переменная  ссылается на переменную внешней функции, при этом (1) не происходит присвоения значения этой переменной или же (2) для переменной внутри функции прописано `nonlocal` выражение.<br>
• `Global`: переменная объявлена на верхнем уровне программы (не внутри функции или метода), и на нее могут быть отсылки из любой части программы. Переменная внутри функции может ссылаться на глобальную переменную, если (1) ей не присваивается значение ни в этой функции, ни во внешней к ней (при наличии), или (2) для переменной прописано `global` выражение.<br>
• `Built-in`: переменная, которая не была нигде объявлена, является встроенной (например, `list`, `print`, `divmod`). 

Если используемая переменная является nonlocal (enclosed) или global, хорошей практикой является объявить ее как таковую, даже если ей не присваивается значение внутри этой функции.

Область видимости переменной, если она отличная от дефолтной, объявляется путем следующего синтаксиса:

In [ ]:
global var1, …, varN 
nonlocal var1, …, varN

Такое объявление должно предшествовать любому использованию переменной.

Как только функция заканчивает работу, локальная переменная уничтожается

In [ ]:
def f():
    print(a)
    if False:
        a = 0
a = 1
f()

In [ ]:
def f():
    global a #дает доступ к глобальным функциям, даже если не были определены
    a = 1
    print(a, end=' ')
a = 0
f()
print(a)

Хотелось бы заострить внимение на том, что **использование глобальных переменных внутри функций - очень плохо!**

Почему?

Избегайте глобальных переменных в функции (добавляйте дополнительные параметры и тд)

NB: иногда к глобальным переменным добавляют префикс _ в качестве соглашения

#### non-local

Локальной могут быть не только переменные, но и функции, и non-local часто возникает в этом контексте. По умолчанию, вложенная функция имеет доступ к, но не может менять переменные внешней функции. Однако если вложенная функция объявляет переменную nonlocal, она может как получить к ней доступ, так и изменить.

In [ ]:
global_greet = "Hello"
def outer_function():
    nonlocal_greet = "Hi"
    def inner_function():
        nonlocal nonlocal_greet
        global global_greet
        nonlocal_greet = "Hi"
        global_greet = "Hey"
    inner_function()
    print(nonlocal_greet, global_greet)
outer_function()

### del

В целом, чтобы удалить объект, нужно вызвать деструктор. В Питоне его роль выполняет метод `__del__()`. В Питоне реализован автоматический сборщик мусора, который очищает память от ненужных объектов, вызывая del, когда все ссылки на объект удаляются. Можно вызвать его и принудительно. Отвязанный от ссылок объект == удаленный.

## Функциональное программирование

### Немного теории

Вопрос: что знаете про ФП?

Определять можно по-разному, но как правило, определения пересекаются по следующим аспектам:<br>

• Функции являются объектами, и к ним можно относиться как к любым другим значениям.<br>
• Переменные поддерживают лишь единственное присвоение (являются константами) - после присвоения значения они далее не меняются.<br>
• Функции являются чистыми - значение, возвращаемое функцией, зависит только от переданных ей аргументов. Данный подход исключает возможность использования глобальных переменных или любых других внешних факторов, таких как system clock.<br>

In [ ]:
def remove_last_item(mylist):
    """Removes the last item from a list."""
    mylist.pop(-1)
    
def butlast(mylist):
"""Like butlast in Lisp; returns the list without the last element."""
    return mylist[:-1]

Преимущества:<br>

• Формальная доказуемость<br>
• Модульность<br>
• Краткость<br>
• Абсолютно безопасные параллельные вычисления<br>
• Легко тестировать - на тот же вход одинаковый выход

Рассмотрим поподробнее первый аспект определения - что функции являются объектами. 

Следующая функция ищет максимальное значение в (непустом) списке:

In [ ]:
def biggest(values): 
    big = values[0]
    for v in values: 
        if v > big: 
            big = v
    return big

Как мы выяснили, `>` работает также и для строк. Однако для всего остального (например, найти самую длинную строку) нам придется задавать новую функцию, причем почти идентичную старой

Вместо этого мы можем заменить `>` обобщенным тестом:

In [ ]:
def most(values, more): 
    best = values[0] 
    for v in values:
        if more(v, best): 
            best=v
    return best

def larger(a, b): 
    return a > b

И вызвать функцию most следующим образом:

In [ ]:
most([1, 6, 1, 8, 0], larger)

или так, в случае нахождения самой длинной строки:

In [ ]:
def longer(a, b):
    return len(a) > len(b)

most(["a", "generic", "list"], longer)

В результате может возникнуть множество небольших функций наподобие `larger` и `longer`. Питон располагает конструкцией, которую можно назвать функцией-литералом, анонимной функцией, или лямбда-функцией (так как исторически вводятся при помощи ключевого слова `lambda`). Они предназначены для однократного использования там, где были введены. Сигнатура состоит из `lambda`, переменные (любое количество), двоеточие, и одно-единственное выражение. В данном случае результат выражения имеет булев тип, но мог бы иметь любой другой:

In [ ]:
lambda a, b: len(a) > len(b)

Используем теперь следующим образом:

In [ ]:
print(most(["a", "generic", "list"], lambda a, b: len(a) > len(b)))

#### Деанонимизация

При желании анонимную функцию можно легко "деанонимизировать":

In [ ]:
square = lambda x: x**2
square(5)

In [ ]:
func = lambda x, y, z: x + y + z
func(2, 3, 4)

Задание: напишите анонимную функцию, которая возвращает несколько значений

In [ ]:
###YOUR CODE HERE###

Задание: отсортируйте массив по возрастанию абсолютных значений при помощи анонимной функции

In [ ]:
a = [1, 3, -2, 4, -5]

In [ ]:
###YOUR CODE HERE###

### В этом же контексте поговорим про рекурсию...

Рекурсия - запуск функции из самой себя.

In [ ]:
n = int(input())
fact = 1
i = 2
while i <= n:
    fact *= i
    i += 1
    print(fact)

In [ ]:
def factorial(n):
    if n == 0 :
        return 1
    return n*factorial(n - 1)

n = int(input())
print(factorial(n))

Задание: реализуйте печать цифр от 0 до 10 при помощи рекурсии (без использования `for`)

In [ ]:
###YOUR CODE HERE###

### ... и декораторы

Декораторы позволяют модифицировать поведение объекта, "обернув" ее в функцию. Объектом в нашем контексте выступает другая функция. Функция-декоратор, таким, образом, принимает исходную в качестве аргумента, и возвращает новую функцию, которая заменяет ее. Для удобства используется синтаксис `@decorator`, размещаемый непосредственно перед декорируемой функцией; decorator при этом - название функции-декоратора.

In [ ]:
def my_decorator(fn):
    def wrapper():
        print("entering the function...")
        fn()
    print("exiting the function...")
    return wrapper

@my_decorator
def my_function():
    print("inside the function...")

my_function()

Декораторы можно применять последовательно, "наслаивая" друг на друга:

In [ ]:
def decorator_1(fn):
    def wrapper():
        print("entering decorator 1...")
        fn()
        print("exiting decorator 1...")
    return wrapper

def decorator_2(fn):
    def wrapper():
        print("entering decorator 2...")
        fn()
        print("exiting decorator 2...")
    return wrapper

def decorator_3(fn):
    def wrapper():
        print("entering decorator 3...")
        fn()
        print("exiting decorator 3...")
    return wrapper

@decorator_1
@decorator_2
@decorator_3
def my_function():
    print("inside the function.")

my_function()

В конечном итоге необходимо понимать, что декораторы являются "синтаксическим сахаром", делая код элегантнее, но не добавляя новой функциональности:

In [ ]:
def dec(fn):
    def wrapper():
        fn()
    return wrapper

# конструкция ниже...
@dec
def fn():
    pass

# эквивалентна следующей...
fn = dec(fn)

### Итераторы и генераторы

Вопрос: в чем разница между `iterable` и `iterator`?

Ответ: `iterable` (перебираемые объекты) - это объекты, к которым можно получить последовательный доступ. Например, `range`:

In [ ]:
print(tuple(range(10)))

Задание: напишите проверку на то, является ли объект перебираемым

In [ ]:
arr = ['i', 'love', 'working', 'with', 'Python']
b = 45.7

###YOUR CODE###

### Итераторы

Итератор - объект, который отслеживат свое местонахождение в итерируемым объекте, и по запросу выдает следующее значение. Можно создать итератор по списку; далее можем вызывать `next(it)`, пока не получим исключение `StopIteration`

In [ ]:
it = iter([2, 3, 5])

Чтобы вручную реализовать итерацию при помощи итератора: 

• Класс iterable объекта должен содержать метод  `__iter__` для создания и возрата нового итератора

• Класс итератора должен содержать два метода:<br> 
• метод `__init__` , который принимает `iterable` в качестве аргумента и производит остальные необходимые при инициализации вычисления;<br>
• метод `__next__` чтобы найти или вычислить следующее значение. Когда возвращать больше нечего, должен выбрасывать исключение `StopIteration`.

In [ ]:
class MyList(): 
    def __init__(self, ls): 
        self.ls = ls 
        
    def __iter__(self): 
        return Reverser(self.ls) 

class Reverser(): 
    def __init__(self, ls): 
        self.ls = ls 
        self.index = len(self.ls) 
    
    def __next__(self): 
        self.index = self.index - 1
        if self.index >= 0: 
            return self.ls[self.index] 
        raise StopIteration

Как мы видим, итератор использует свои внутренние переменные для отслеживания своей позиции относительно объекта, по которому мы итерируемся. Поэтому он никак не модифицирует этот самый объект, и на нем может быть запущено сразу несколько итераторов параллельно.

В цикле for может быть использована с любым итератором. В цикле `for`, исключение `StopIteration` не приводит к ошибке, а просто приводит к выходу из цикла.

In [ ]:
ls = MyList([1, 2, 3, 4]) 

for e in ls: 
    print(e)

Задание со звездочкой: реализуйте цикл for при помощи итератора

In [ ]:
###YOUR CODE###

Итератор не может быть "переиспользован", или "перезагружен" - после того, как он выбросит `StopIteration`, мы можем только создать новый.

### Генераторы

Генераторы похожи на итераторы. Он генерирует значения по-одному, но при этом не обязательно привязан к какому-то объекту.

Генератор можно задать почти как списочное включение, вместо квадратных скобок используя круглые, и использовать везде, где можно использовать итератор:

In [ ]:
word = 'generator'
gen = (c for c in word if c in 'aeiou')
for i in gen:
    print(i, end=' ')

Как и итератор, после полного цикла он принимает пустое значение и больше ничего не вернет.

In [ ]:
for i in gen:
    print(i, end=' ')

Смысл генератора чуть более проясняется на примере написания функций, которые ведут себя генераторы. В них вместо return мы используем yield

In [ ]:
def powers_of_two():
    n = 2
    for i in range(0, 5):
        yield n
        n *= 2

При вызове функции возвращается не число, а генератор, который мы затем можем использовать

In [ ]:
gen = powers_of_two()
for n in gen:
    print(n)

Генерируем следующее значение, и обрабатываем исключение

In [ ]:
gen = powers_of_two()
while True:
    try:
        print(next(gen))
    except StopIteration:
        break

Что тут происходит:<br>
• Вызов `gen = powers_of_two()` возвращает генератор и кладет в переменную gen.<br>
• Первый вызов `next(gen)` исполнит код генератора до `yield` и вернет необходимое значение, как обычный `return`. Однако вдобавок генератор запомнит свое состояние.<br>
• Следующий вызов `next(gen)` вернет исполнение в генераторе с предыдуюего состояния, то есть сразу после `yield`. Все значения локальных переменных будут восстановлены - как будто бы `yield` не происходил. В нашем примере, цикл for продолжит исполнение.<br>
• Можно использовать `yield` несколько раз.<br>
• В конце - все так же `StopIteration`.

#### Генераторы и память

In [ ]:
import sys

# проверяем расход памяти
def memory_size(_, code):
    size = sys.getsizeof(code)
    return f'{_}: allocated memory is {size} bytes'

print(memory_size('generator', (num**2 for num in range(10000))))
print(memory_size('list comprehension', [num**2 for num in range(10000)]))

Генераторы позволяют нам работать с большими датасетами с минимальными затратами памяти.

### Прочие функциональные инструменты

Питон располагает рядом встроенных инструментов:

`map(function, iterable)` применяет функцию к каждому элементу в перечиляемом объекте и возвращает перечисляемый объект `map`

In [ ]:
print(map(lambda x: x + "bzz!", ["I think", "I'm good"]))
list(map(lambda x: x + "bzz!", ["I think", "I'm good"]))

`filter(function or None, iterable)` отбирает элементы из перечисляемого объекта на основании результата применения к ним функции, и возвращает перечисляемый объект `filter`

In [ ]:
print(filter(lambda x: x.startswith("I "), ["I think", "I'm good"]))
list(filter(lambda x: x.startswith("I "), ["I think", "I'm good"]))

`zip(iter1 [,iter2 [...]])` принимает на вход последовательности одной длины, и поэлементно собирает их в последовательность кортежей. Полезно, например, если нам нужно объединить список ключей и список значений в словарь. Возвращает перечисляемый объект `zip`

In [ ]:
keys = ["foobar", "barzz", "ba!"]
print(zip(keys, map(len, keys)))
print(list(zip(keys, map(len, keys))))
print(dict(zip(keys, map(len, keys))))

`functools.reduce(binaryFunction, iterable)` применяет бинарную функцию к первым двум элементам перечисляемого объекта. Затем операция последовательно повторяется по отношению к результату применения функции и следующему элементу, пока перечисляемый объект не свернется в одно результирующее значение. (NB: `reduce` необходимо импортировать из библиотеки `functools`).

In [ ]:
my_list = [3,1,4,1,6]
from functools import reduce
reduce(lambda x,y:x+y,my_list)

`any(iterable)` и `all(iterable)` возвращают булево значение в зависимости от значений на элементах перечисляегого объекта. Они эквивалентны следующему:

In [ ]:
def all(iterable):
    for x in iterable:
        if not x:
            return False
    return True

In [ ]:
def any(iterable):
    for x in iterable:
        if x:
            return True
    return False

Их удобно применять в том случае, если нужно проверить, соответствуют ли все / хоть один элемент в перечисляемом объекте условию:

In [ ]:
mylist = [0, 1, 3, -1]
if all(map(lambda x: x > 0, mylist)):
    print("All items are greater than 0")
if any(map(lambda x: x > 0, mylist)):
    print("At least one item is greater than 0")

### Вернемся к списочным включениям в этом контексте

Как мы выяснили, списочные включения - это отличный способ задания списков. Стоит отметить, что списки, которые заданы при помощи включений, ни в чем не опираются на состояние программы, и потому вписываются в парадигму функционального программирования (и именно в силу этого является столь элегантным и легким к восприятию приемом).

Задание: напишите эквивалент `filter` и `map` при помощи списочного включения:

`map`

In [ ]:
###YOUR CODE###

`filter`

In [ ]:
###YOUR CODE###

### Обратная сторона лямбды

Подоплека: когда мы работаем со списками, часто встает задача найти первый элемент, удовлетворяющий определенному условию. Обычно эта задача решается при помощи конструкции наподобие следующей:

In [ ]:
def first_positive_number(numbers):
    for n in numbers:
        if n > 0:
    return n

В функциональном стиле можно переписать так:

In [ ]:
def first(predicate, items):
    for item in items:
        if predicate(item):
    return item

first(lambda x: x > 0, [-1, 0, 1, 2])

In [ ]:
# Less efficient
list(filter(lambda x: x > 0, [-1, 0, 1, 2]))[0]
# Efficient
next(filter(lambda x: x > 0, [-1, 0, 1, 2]))

Вопрос: какую вы видите проблему с первым вариантом?

Можно немного упростить себе жизнь, воспользовавшись небольшой полезной библиотекой first:

In [ ]:
from first import first

a = first([0, False, None, [], (), 42])
b = first([-1, 0, 1, 2])
c = first([-1, 0, 1, 2], key=lambda x: x > 0)
print(a, b, c)

Здесь мы используем именованный аргумент `key` с тем, чтобы передать в него функцию, которая поочередно оценивает каждый аргумент на соответствие условию.

Как можно увидеть, в семинаре мы повсеместно используем для подобных случаев `lambda`. Вообще говоря, `lambda` была включена в Python в первую очередь для удобства использования функциональных `map()` и `filter()`, который в противном случае потребовали бы определять новую функцию каждый раз когда нужно проверить какое-то новое условие.

In [ ]:
import operator
from first import first

def greater_than_zero(number):
    return number > 0

first([-1, 0, 1, 2], key=greater_than_zero)

Код выше работает аналогично предыдущей ячейке, но он гораздо более громоздкий. Если, например, нам понадобится найти первое число в последовательности, которое превосходит, скажем, 42, то нам бы пришлось определять соответствующую отдельную функцию вместо того, чтобы определить ее в одной строке с вызовом `first`.

Несмотря на всю полезность в подобных ситуациях, `lambda` не лишена проблем. Самое очевидное, мы не можем таким образом передать в ключ функцию длиной более одной строчки кода. Однако означает ли это, что в этой ситуации у нас нет другого выхода, кроме как определять отдельную функцию? Не совсем.

`functools.partial` предоставляет альтернативу лямбда-выражению, причем даже более гибкую. Эта конструкция позволяет создать функцию-обертку с интересной особенностью: вместо того, чтобы менять поведение функции, мы взамен меняем аргументы, которые она получает на вход. Например:

In [ ]:
from functools import partial
from first import first

def greater_than(number, min=0):
    return number > min

first([-1, 0, 1, 2], key=partial(greater_than, min=42))

Теперь наша функция `greater_than` по умолчанию работает как прежняя, но вдобавок мы можем указать значение, с которым мы сравниваем передаваемый в нее аргумент. В данном случае, мы передаем в `functools.partial` нашу исходную функцию и значение, которым мы переопределим `min`, и в результате получим новую функцию, которая сравнивает числа на входе с 42, ровно как мы и хотели бы. Другими словами, мы можем задать функцию, и затем кастомизировать ее при помощи `functools.partial` так, как нам необходимо в данной ситуации.

Строго говоря, в данном конкретном случае наша запись все еще избыточна, ведь все, что мы делаем, это сравнение двух чисел. В Питон присутствует специальный модуль `operator` для подобного рода простых операций:

In [ ]:
import operator
from functools import partial
from first import first

first([-1, 0, 1, 2], key=partial(operator.le, 0))

Как мы можем убедиться, `functools.partial` работает и с позиционными аргументами тоже. В данном примере `operator.le(a, b)` принимает на вход два числа и возвращает булево значение в зависимости от того, больше или равно первое второму или нет. Ноль, который мы передаем в `functools.partial`, уходит в переменную `a`, в то время как то, что уходит в функцию, которую мы получаем на выходе из `functools.partial`, уходит в `b`. Таким образом, используя `le` (а не `ge`, как могло бы показаться), наш пример работает должным образом без необходимости в лямбда-выражении и задании каких-либо дополнительных функций.

`functools.partial` особенно полезна в качестве замены `lambda` - которую, к слову сказать, даже планировали убрать из третьей версии Пайтон! - и считается предпочтительной альтернативой. Лямбда-выражения являются некоторой аномалией ввиду ограничения на длину в одну стоку. С другой стороны, `functools.partial` предоставляет удобную обертку вокруг исходной функции.

Вообще говоря, модуль `itertools` в составе Python Standard Library содержит целый ряд полезных функций, которые неплохо держать в уме. Очень часто можно встретить примеры, когда разработчики прописывают свои версии данных функций... называя вещи своими именами, изобретают велосипед, когда есть отличные готовые реализации:

• `chain(*iterables)` итерация по элементам перечисляемого объекта без явного построения промежуточного списка всех элементов<br>
• `combinations(iterable, r)` генерирует все комбинации длины `r` из данного перечисляемого объекта<br>
• `compress(data, selectors)` применяет булеву маску из `selectors` к данным и возвращает только те значения из них, где соответствующие элемент селектора истинен<br>
• `count(start, step)` генерирует бесконечную последовательность значений, начиная со `start` и увеличиваясь на `step` на каждом вызове<br>
• `cycle(iterable)` циклически перебирает элементы в перечисляемом объекте<br>
• `dropwhile(predicate, iterable)` отфильтровывает элементы перечисляемого объекта с начала и до момента когда предикат оценится как ложный<br>
• `groupby(iterable, keyfunc)` создает итератор, который группирует элементы по результату, который возвращает на них функция `keyfunc`<br>
• `permutations(iterable[, r])` возвращает последовательные перестановки элементов перечисляемого объекта длины `r`<br>
• `product(*iterables)` возвращает перечисляемый объект декартова произведения перечисляемых объектов без задействования вложенных циклов<br>
• `takewhile(predicate, iterable)` возвращает элементы перечисляемого объекта с начала и до момента когда предикат оценится как ложный<br>

Наибольшую мощь эти функции приобретают в комбинации с модулем `operator`; сочетание `itertools` и `operator` может заменить собой `lambda` практически во всех ситуациях:

In [58]:
import itertools

a = [{'foo': 'bar'}, {'foo': 'bar', 'x': 42}, {'foo': 'baz', 'y': 43}]

import operator

print(list(itertools.groupby(a, operator.itemgetter('foo'))))
[(key, list(group)) for key, group in list(itertools.groupby(a, operator.itemgetter('foo')))]

[('bar', <itertools._grouper object at 0x7fd225800640>), ('baz', <itertools._grouper object at 0x7fd225800d30>)]


[('bar', []), ('baz', [])]

В данном случае можно было бы воспользоваться конструкцией `lambda x: x['foo']`, однако использование `operator` позволяет совсем отказаться от лямбда-выражения.